In [1]:
%cd "/data/ai_club/team_C/"

/data/ai_club/team_C


In [2]:
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from spectral import *
from PIL import Image, ImageFile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [3]:
#Data Files
data_train = np.load('CNN/spectral_lymph_tile_array_3_9_22_left_update_1.npy')
non_data_train = np.load('CNN/spectral_non_tile_array_3_9_22_left_update_1.npy')
data_test = np.load('CNN/spectral_lymph_tile_array_3_9_22_right_update_1.npy')
non_data_test = np.load('CNN/spectral_non_tile_array_3_9_22_right_update_1.npy')

data_stack_train = np.vstack((data_train, non_data_train)) # One array of all training data
data_stack_test = np.vstack((data_test, non_data_test))   # One array of all test data
labels_train = np.append(np.ones(data_train.shape[0]), np.zeros(non_data_train.shape[0])) 
labels_test = np.append(np.ones(data_test.shape[0]), np.zeros(non_data_test.shape[0]))

X_train = data_stack_train[:, :]
y_train = labels_train[:]
X_test =  data_stack_test[:, :]
y_test = labels_test[:]
print("Data Loaded")


Data Loaded


In [31]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=5, verbose=2, mode='auto',
    baseline=None, restore_best_weights=True
)

In [35]:
#Model
model=Sequential()
model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu",input_shape=(17,17,394)))
model.add(Conv2D(filters=16,kernel_size=3,padding="same",activation="relu"))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation="sigmoid"))  
print(model.summary())
optimizer = tf.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #compile Model
history=model.fit(X_train,y_train,epochs=25, validation_data=(X_test,y_test), callbacks=[callback]) #train model

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 17, 17, 128)       454016    
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 17, 17, 16)        18448     
_________________________________________________________________
flatten_19 (Flatten)         (None, 4624)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 512)               2368000   
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 513       
Total params: 2,840,977
Trainable params: 2,840,977
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
595/595 [==============================] - 23s 39ms/step - loss: 0.2243 - accuracy: 0.9041 - 

In [36]:
#Loading Spectral Array and creating cropped spectral array 
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True
spectral_img = open_image('ov-63-hd-16ca.hdr')

In [37]:
def predict(size, x_start, y_start,x_num,y_num):
    for x_times in range(x_num):
        for y_times in range(y_num):
            spectral_shape=spectral_img.shape
            spectral_2d_shape=spectral_shape[:2]
            mask = np.zeros((spectral_2d_shape))
            mask[(x_start+(size*(x_times))):x_start+(size*(x_times+1)),y_start+(size*(y_times)):y_start+(size*(y_times+1))]=1
            #Create 17x17 tiles
            
            # Tile radius (1 = 3x3, 2 = 5x5, 3 = 7x7, etc.)
            tr = 8
            # Finding (y,x) tuple for where the mask is True
            y, x = np.where(mask == True)
            
            # Declaring pixel array
            tw = 1 + (tr * 2)
            test_spectral_result = np.empty((len(y), tw, tw, 394))
            
            # # Append the pixels in the spectral image to the pixel array for every (y,x) and (ny,nx) tuple
            print("Creating spectral array...")
            for index, point in enumerate(zip(y, x)):
                px, py = point[1], point[0]
                #print((index/float(len(y)))*100) # If you want to see progress for large tables
                if(index%1000 ==0 ): print((index/float(len(y)))*100, "%") # Progress
                test_spectral_result[index] = spectral_img[py-tr:py+tr+1, px-tr:px+tr+1]
            print("predicting")    
            predictions=model.predict(test_spectral_result)
            print("Saving")
            predictions=predictions.reshape(size,size)
            plt.imsave('test.png', predictions , cmap=plt.cm.gray)
            #Make a mask file out of this file for easy comparison to hyperspectral file
            output_mask = np.zeros((17912, 20292))
            output_mask[(x_start+(size*(x_times))):x_start+(size*(x_times+1)),y_start+(size*(y_times)):y_start+(size*(y_times+1))]=predictions
            ##print(output_mask.shape)
            fig = plt.figure(figsize = (10,10))
            # Save the image:
            image_file = Image.fromarray(output_mask, mode="1")
            filename="predictions_mask_x_" + str(((x_start+(size*(x_times))))) +"_y_" +str(((y_start+(size*(y_times))))) +"_size_" +str(size) 
            print(filename)
            #Set colormap https://kbkb-wx-python.blogspot.com/2015/12/python-transparent-colormap.html
            colors = [(1,1,1,c) for c in np.linspace(0,1,100)]
            cmapwhite = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=255) #increase this if running again
            plt.imsave(f"tiled_output/4_28_22/{filename}_white.png", output_mask , cmap=cmapwhite)#cmap=plt.cm.gray)

In [38]:
predict(250,9000,15750,5,5) # backwards x,y (row, column)

Creating spectral array...
0.0 %
1.6 %
3.2 %
4.8 %
6.4 %
8.0 %
9.6 %
11.200000000000001 %
12.8 %
14.399999999999999 %
16.0 %
17.599999999999998 %
19.2 %
20.8 %
22.400000000000002 %
24.0 %
25.6 %
27.200000000000003 %
28.799999999999997 %
30.4 %
32.0 %
33.6 %
35.199999999999996 %
36.8 %
38.4 %
40.0 %
41.6 %
43.2 %
44.800000000000004 %
46.400000000000006 %
48.0 %
49.6 %
51.2 %
52.800000000000004 %
54.400000000000006 %
56.00000000000001 %
57.599999999999994 %
59.199999999999996 %
60.8 %
62.4 %
64.0 %
65.60000000000001 %
67.2 %
68.8 %
70.39999999999999 %
72.0 %
73.6 %
75.2 %
76.8 %
78.4 %
80.0 %
81.6 %
83.2 %
84.8 %
86.4 %
88.0 %
89.60000000000001 %
91.2 %
92.80000000000001 %
94.39999999999999 %
96.0 %
97.6 %
99.2 %
predicting
Saving
predictions_mask_x_9000_y_15750_size_250
Creating spectral array...
0.0 %
1.6 %
3.2 %
4.8 %
6.4 %
8.0 %
9.6 %
11.200000000000001 %
12.8 %
14.399999999999999 %
16.0 %
17.599999999999998 %
19.2 %
20.8 %
22.400000000000002 %
24.0 %
25.6 %
27.200000000000003 %
28.7

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


predictions_mask_x_10000_y_15750_size_250
Creating spectral array...
0.0 %
1.6 %
3.2 %
4.8 %
6.4 %
8.0 %
9.6 %
11.200000000000001 %
12.8 %
14.399999999999999 %
16.0 %
17.599999999999998 %
19.2 %
20.8 %
22.400000000000002 %
24.0 %
25.6 %
27.200000000000003 %
28.799999999999997 %
30.4 %
32.0 %
33.6 %
35.199999999999996 %
36.8 %
38.4 %
40.0 %
41.6 %
43.2 %
44.800000000000004 %
46.400000000000006 %
48.0 %
49.6 %
51.2 %
52.800000000000004 %
54.400000000000006 %
56.00000000000001 %
57.599999999999994 %
59.199999999999996 %
60.8 %
62.4 %
64.0 %
65.60000000000001 %
67.2 %
68.8 %
70.39999999999999 %
72.0 %
73.6 %
75.2 %
76.8 %
78.4 %
80.0 %
81.6 %
83.2 %
84.8 %
86.4 %
88.0 %
89.60000000000001 %
91.2 %
92.80000000000001 %
94.39999999999999 %
96.0 %
97.6 %
99.2 %
predicting
Saving
predictions_mask_x_10000_y_16000_size_250
Creating spectral array...
0.0 %
1.6 %
3.2 %
4.8 %
6.4 %
8.0 %
9.6 %
11.200000000000001 %
12.8 %
14.399999999999999 %
16.0 %
17.599999999999998 %
19.2 %
20.8 %
22.40000000000000

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>